OK, so let's finally get all calls into PLINK:

In [11]:
cd /data/sudregp/cnv/penncnv/results

rm denovo.jointcnv inherited.jointcnv all.jointcnv 2>/dev/null
while read sample; do
    triocnv=${sample}'_adjusted.jointcnv'
    grep mother ${triocnv} > mom_snps;
    grep father ${triocnv} > dad_snps;
    cat mom_snps dad_snps > parent_snps;
    grep offspring ${triocnv} > kid_snps;
    for snp in `cut -d' ' -f 1 kid_snps`; do
        if ! grep -q $snp parent_snps; then
            grep $snp kid_snps >> denovo.jointcnv
        else
            grep $snp kid_snps >> inherited.jointcnv
        fi;
        grep $snp kid_snps >> all.jointcnv
   done
   rm *_snps;
done < ../good_kids_joint_qc20171215.txt

In [12]:
wc -l denovo.jointcnv
wc -l inherited.jointcnv
wc -l all.jointcnv

1364 denovo.jointcnv
518 inherited.jointcnv
1882 all.jointcnv


Creating PLINK files:

In [14]:
cd /data/sudregp/cnv/penncnv/
chmod +x penncnv_to_plink.pl
cd results/
cp /data/NCR_SBRB/simplex/simplex.ped ../
cd ..
sed -i -e "s/CLIA/InfiniumExome\/CLIA/g" simplex.ped
sed -i -e "s/CCGO/HumanExome\/CCGO/g" simplex.ped
cd results/
../penncnv_to_plink.pl -i denovo.jointcnv -o denovo.cnv -f ../simplex.ped
../penncnv_to_plink.pl -i inherited.jointcnv -o inherited.cnv -f ../simplex.ped
../penncnv_to_plink.pl -i all.jointcnv -o all.cnv -f ../simplex.ped

I'll play with the all.cnv file, but we can easily expand it to the denovo.cnv or inherited.cnv file later.

In [21]:
module load plink/1.07

plink --cnv-list all.cnv --cnv-make-map --noweb --out all


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all.log ]
Analysis started: Mon Dec 18 15:02:14 2017

Options in effect:
	--cnv-list all.cnv
	--cnv-make-map
	--noweb
	--out all


Reading segment list (CNVs) from [ all.cnv ]
Writing new MAP file to [ all.cnv.map ]
Wrote 3733 unique positions to file

Analysis finished: Mon Dec 18 15:02:14 2017



In [22]:
plink --map all.cnv.map --fam ../simplex.ped --cnv-list all.cnv --out all \
    --noweb --1 --cnv-check-no-overlap


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ all.log ]
Analysis started: Mon Dec 18 15:02:28 2017

Options in effect:
	--map all.cnv.map
	--fam ../simplex.ped
	--cnv-list all.cnv
	--out all
	--noweb
	--1
	--cnv-check-no-overlap

Reading marker information from [ all.cnv.map ]
3733 (of 3733) markers to be included from [ all.cnv.map ]
Reading individual information from [ ../simplex.ped ]
Reading pedigree information from [ ../simplex.ped ] 
99 individuals read from [ ../simplex.ped ]

OK, at this point I need to perform severla operations to remove and include CNVs. It's easier if we just accumulate all regions in a file, and feed it all to PLINK at once:

* telomeres
* centromeres
* mm
* well-know CNVs

It's possible that one (or more) of these regions are actually significant in our analysis, but for now let's be conservative and remove them as they have been shown to be somehow problematic in previous analysis.

I also figured out how to do the merging trick, so let's do that as well. I just had to output some headers to the PBF file, like this:

In [ ]:
echo -e "SNP\tChr\tPos" > ../signalfile.txt
cut -f 3,2,1 ../InfiniumExome.pfb >> ../signalfile.txt

Note that I'm using the default .2 fraction, but also the .5 they used in the paper.

In [24]:
module load penncnv

while read sample; do
    triocnv=${sample}'_adjusted.jointcnv'
    for frac in .2 .5; do
        clean_cnv.pl --signalfile ../signalfile.txt combineseg $triocnv \
            --fraction $frac > ${triocnv}_comb${frac};
    done;
done < ../good_kids_joint_qc20171215.txt

NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Merging chr6:31391401-31407643 and chr6:31416156-31451279 with marker fraction=0.077 (3694-3690-1) / (3727-3688)  (31416156-31407643-1) / (31451279-31391401)
NOTICE: Merging chr6:31391401-31407643 and chr6:31416156-31451279 with marker fraction=0.077 (3694-3690-1) / (3727-3688)  (31416156-31407643-1) / (31451279-31391401)
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Merging chr6:31391401-31407643 and chr6:31416156-31451279 with marker fraction=0.077 (3694-3690-1) / (3727-3688)  (31416156-31407643-1) / (31451279-31391401)
NOTICE: Merging chr6:31391401-31407643 and chr6:31416156-31451279 with marker fraction=0.077 (3694-3690-1) / (3727-3688)  (31416156-31407643-1) / (31451279-31391401)
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Merging chr9:17273731-17309120 and chr9:17415825-17464622 with marker fr

NOTICE: Merging chr19:19822451-22364389 and chr19:23837365-23927839 with marker fraction=0.095 (6741-6726-1) / (6746-6598)  (23837365-22364389-1) / (23927839-19822451)
NOTICE: Merging chr19:19822451-23927839 and chr19:30496559-30505869 with marker fraction=0.235 (6795-6746-1) / (6802-6598)  (30496559-23927839-1) / (30505869-19822451)
NOTICE: Merging chr19:53086062-53117817 and chr19:53303254-53454387 with marker fraction=0.205 (12669-12653-1) / (12703-12630)  (53303254-53117817-1) / (53454387-53086062)
NOTICE: Merging chr1:248569417-248685858 and chr1:248845110-249212429 with marker fraction=0.141 (24664-24651-1) / (24708-24623)  (248845110-248685858-1) / (249212429-248569417)
NOTICE: Merging chr14:45618055-45628478 and chr14:45644409-45654570 with marker fraction=0.263 (2394-2383-1) / (2415-2377)  (45644409-45628478-1) / (45654570-45618055)
NOTICE: Merging chr14:92195816-92278829 and chr14:92439059-92441066 with marker fraction=0.444 (6070-6057-1) / (6075-6048)  (92439059-92278829-1) 

NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Merging chr16:1841389-1842516 and chr16:1884281-1918157 with marker fraction=0.465 (1103-1082-1) / (1112-1069)  (1884281-1842516-1) / (1918157-1841389)
NOTICE: Merging chr9:17273731-17309120 and chr9:17415825-17464622 with m

NOTICE: Merging chr5:171338-171559 and chr5:191675-192294 with marker fraction=0.500 (56-46-1) / (60-42)  (191675-171559-1) / (192294-171338)
NOTICE: Merging chr13:96508566-96553101 and chr13:96599295-96743177 with marker fraction=0.250 (3288-3278-1) / (3300-3264)  (96599295-96553101-1) / (96743177-96508566)
NOTICE: Merging chr1:225140459-225155380 and chr1:225373072-225452983 with marker fraction=0.514 (22238-22219-1) / (22243-22208)  (225373072-225155380-1) / (225452983-225140459)
NOTICE: Merging chr9:15723682-15729695 and chr9:15777640-15971815 with marker fraction=0.355 (1020-1008-1) / (1031-1000)  (15777640-15729695-1) / (15971815-15723682)
NOTICE: Merging chr9:17298289-17309120 and chr9:17409391-17464622 with marker fraction=0.394 (1098-1084-1) / (1113-1080)  (17409391-17309120-1) / (17464622-17298289)
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Total of 243345 records are read from ../sig

NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Merging chr14:102963966-102974103 and chr14:103390134-103396961 with marker fraction=0.526 (7176-7165-1) / (7182-7163)  (103390134-102974103-1) / (103396961-102963966)
NOTICE: Merging chr14:102963966-103396961 and chr14:103568488-103574001 with marker fraction=0.439 (7208-7182-1) / (7220-7163)  (103568488-103396961-1) / (103574001-102963966)
NOTICE: Merging chr14:102963966-103574001 and chr14:104000888-104000956 with marker fraction=0.378 (7258-7220-1) / (7261-7163)  (104000888-103574001-1) / (104000956-102963966)
NOTICE: Merging chr10:91470828-91477454 and chr10:91492754-91503578 with marker fraction=0.286 (5356-5343-1) / (5378-5336)  (91492754-91477454-1) / (91503578-91470828)
NOTICE: Merging chr10:117024753-117045897 and chr10:117226750-117228778 with marker fraction=0.462 (8080-8073-1) / (8084-8071)  (117226750-117045897-1) / (117228778-117024753)
NOTICE: Merging chr9:17273731-17309120 and chr9:17415825-1746462

NOTICE: Merging chr20:61287939-61292481 and chr20:61340732-61386238 with marker fraction=0.343 (5802-5789-1) / (5813-5778)  (61340732-61292481-1) / (61386238-61287939)
NOTICE: Merging chr20:61981303-61981603 and chr20:62038277-62168633 with marker fraction=0.176 (6040-6033-1) / (6064-6030)  (62038277-61981603-1) / (62168633-61981303)
NOTICE: Merging chr20:61981303-62168633 and chr20:62178461-62187744 with marker fraction=0.277 (6083-6064-1) / (6095-6030)  (62178461-62168633-1) / (62187744-61981303)
NOTICE: Merging chr20:61981303-62187744 and chr20:62193478-62202150 with marker fraction=0.115 (6115-6095-1) / (6195-6030)  (62193478-62187744-1) / (62202150-61981303)
NOTICE: Merging chr20:61981303-62202150 and chr20:62328236-62329672 with marker fraction=0.238 (6249-6195-1) / (6253-6030)  (62328236-62202150-1) / (62329672-61981303)
NOTICE: Merging chr20:61981303-62329672 and chr20:62421480-62494241 with marker fraction=0.117 (6286-6253-1) / (6303-6030)  (62421480-62329672-1) / (62494241-61

NOTICE: Merging chr21:47421947-47581949 and chr21:47717471-47717537 with marker fraction=0.436 (2628-2548-1) / (2631-2450)  (47717471-47581949-1) / (47717537-47421947)
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Merging chr1:18807521-18809717 and chr1:19062163-19062494 with marker fraction=0.139 (3024-3018-1) / (3029-2993)  (19062163-18809717-1) / (19062494-18807521)
NOTICE: Merging chr14:104624263-104646102 and chr14:105054085-105054213 with marker fraction=0.158 (7382-7375-1) / (7388-7350)  (105054085-104646102-1) / (105054213-104624263)
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Merging chr19:52496263-52497943 and chr19:52537004-52538193 with marker fraction=0.318 (12454-12439-1) / (12467-12423)  (52537004-52497943-1) / (52538193-52496263)
NOTICE: Merging chr1:18807521-18809717 and chr1:19062163-19062494 with marker fraction=0.139 (3024-3018-1) / (3029-2993)  (19062163-18809717-1) / (19062494-18807521)
NOTICE: Merging chr14:10

NOTICE: Merging chr20:61981303-62200285 and chr20:62421514-62494224 with marker fraction=0.368 (6288-6187-1) / (6302-6030)  (62421514-62200285-1) / (62494224-61981303)
NOTICE: Merging chr7:1015110-1027121 and chr7:1097255-1132095 with marker fraction=0.200 (105-97-1) / (123-88)  (1097255-1027121-1) / (1132095-1015110)
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Merging chr1:17249200-17250945 and chr1:17257829-17273287 with marker fraction=0.214 (2701-2697-1) / (2707-2693)  (17257829-17250945-1) / (17273287-17249200)
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Merging chr1:17249200-17250945 and chr1:17257829-17273287 with marker fraction=0.214 (2701-2697-1) / (2707-2693)  (17257829-17250945-1) / (17273287-17249200)
NOTICE: Total of 243345 records are read from ../signalfile.txt
NOTICE: Merging chr16:50368638-50388719 and chr16:50659417-50664197 with marker fraction=0.182 (5625-5622-1) / (5630-5619)  (50659417-50388719-1) / (5066419

The annoying thing is that clean_cnv removes the family annotation. So, we need to put it back in for our scripts to work:

In [52]:
while read sample; do
    triocnv=${sample}'_adjusted.jointcnv'
    for frac in '.5' '.2'; do
        mom=`grep mother $triocnv | head -1 | awk '{print $5}'`;
        dad=`grep father $triocnv | head -1 | awk '{print $5}'`;
        kid=`grep offspring $triocnv | head -1 | awk '{print $5}'`;
        mv ${triocnv}_comb${frac} junk
        while read line; do
            # if line contains...
            if [[ $line == *${mom}* ]]; then
                echo $line mother >> ${triocnv}_comb${frac};
            elif [[ $line == *${dad}* ]]; then
                echo $line father >> ${triocnv}_comb${frac};
            elif [[ $line == *${kid}* ]]; then
                echo $line offspring >> ${triocnv}_comb${frac};
            fi;
        done < junk;
    done;
done < ../good_kids_joint_qc20171215.txt

In [53]:
for frac in .2 .5; do
    rm denovo.jointcnv_comb${frac} inherited.jointcnv_comb${frac} \
        all.jointcnv_comb${frac} 2>/dev/null
    while read sample; do
        triocnv=${sample}'_adjusted.jointcnv'_comb${frac}
        grep mother ${triocnv} > mom_snps;
        grep father ${triocnv} > dad_snps;
        cat mom_snps dad_snps > parent_snps;
        grep offspring ${triocnv} > kid_snps;
        for snp in `cut -d' ' -f 1 kid_snps`; do
            if ! grep -q $snp parent_snps; then
                grep $snp kid_snps >> denovo.jointcnv_comb${frac}
            else
                grep $snp kid_snps >> inherited.jointcnv_comb${frac}
            fi;
            grep $snp kid_snps >> all.jointcnv_comb${frac}
       done
       rm *_snps;
    done < ../good_kids_joint_qc20171215.txt;
    wc -l denovo.jointcnv_comb${frac}
    wc -l inherited.jointcnv_comb${frac}
    wc -l all.jointcnv_comb${frac}
done


grep: CCGO_800979_adjusted.jointcnv_comb.2: No such file or directory
grep: CCGO_800979_adjusted.jointcnv_comb.2: No such file or directory
grep: CCGO_800979_adjusted.jointcnv_comb.2: No such file or directory
1327 denovo.jointcnv_comb.2
504 inherited.jointcnv_comb.2
1831 all.jointcnv_comb.2
grep: CCGO_800979_adjusted.jointcnv_comb.5: No such file or directory
grep: CCGO_800979_adjusted.jointcnv_comb.5: No such file or directory
grep: CCGO_800979_adjusted.jointcnv_comb.5: No such file or directory
1184 denovo.jointcnv_comb.5
479 inherited.jointcnv_comb.5
1663 all.jointcnv_comb.5


This makes sense, now we have less total CNVs. And apparently CCGO_800979 had no CNVs to begin with, so we're good. Now, let's define all files of things to be removed, and start running them in PLINK: